### 1 - Importar o baco de dados

In [45]:
# Importa LIB
import pandas as pd
import smtplib
import mimetypes
import pathlib
from email.mime.multipart import  MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
# Importa BD
emails = pd.read_excel(r'Bases de Dados\Emails.xlsx')
lojas = pd.read_csv(r'Bases de Dados\Lojas.csv', sep=';', encoding = 'ISO-8859-1')
vendas = pd.read_excel(r'Bases de Dados\Vendas.xlsx')
#------------------------#                      
def vd (BD):
    return display(BD)
#------------------------# 
with open('Senha.txt','r') as arquivo:
    senha = arquivo.read()
    
#vd(emails)
#vd(lojas)
#vd(vendas)

### 2 - Tradar os dados

In [46]:
#Mesclar as lojas com as vendas
vendas = vendas.merge(lojas,on="ID Loja")
#vd(vendas)

### 3 - Criar o arquivo para cada loja

In [47]:
dc_loja = {}
for loja in lojas['Loja']:
    dc_loja[loja] = vendas.loc[vendas['Loja']==loja,:]
    
#vd(dc_loja['Shopping Iguatemi Fortaleza'])
#vd(dc_loja['Shopping Morumbi'])

In [48]:
# Ultimos dias 
dia_indicador = vendas["Data"].max()
print(dia_indicador)

2019-12-26 00:00:00


### 3 - Enviar o email para loja
- Fazer um save de cada loja
- Planilha do dia e do ano

In [49]:
# identificar se a pasta ja existe

caminho_bk = pathlib.Path(r"Backup Arquivos Lojas")
arquivos_pasta_bk = caminho_bk.iterdir()

lista_nome_bk = [arquivo.name for arquivo in arquivos_pasta_bk]
#for arquivo in arquivos_pasta_bk:
#    lista_nome_bk.append(arquivo.name)
#print(lista_nome_bk)

for loja in dc_loja:
    if loja not in lista_nome_bk:
        nova_pasta = caminho_bk / loja
        nova_pasta.mkdir()
    # salvar dentro da pasta
    nome_aq= f'{dia_indicador.day}_{dia_indicador.month}_{dia_indicador.year}_{loja}.xlsx'   
    local_aq = caminho_bk / loja / nome_aq
    dc_loja[loja].to_excel(local_aq) 

### 4 -  Cauculo de indicadores e Enviando o email
- Faturamento
- Divercidade de produtos
- Valor médio por Vendas do dia/ano

In [50]:
# Metas
meta_dia = 1000
meta_ano = 1650000
meta_qtde_dia = 4
meta_qtde_ano = 120
meta_tkmed_dia = 500
meta_tkmed_ano = 500

In [51]:
for loja in dc_loja:
    
    vendas_loja = dc_loja[loja]
    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data']==dia_indicador,:]
    # Faturamento
    ft_ano = vendas_loja['Valor Final'].sum()
    ft_dia = vendas_loja_dia['Valor Final'].sum()
    
    #divercidade de produto
    qtde_ano = len(vendas_loja['Produto'].unique())
    qtde_dia = len(vendas_loja_dia['Produto'].unique())
    
    #ticket medio
    valor_vd = vendas_loja.groupby('Código Venda').sum()
    valor_vd_dia = vendas_loja_dia.groupby('Código Venda').sum()
    tk_ano = valor_vd['Valor Final'].mean()
    tk_dia = valor_vd_dia['Valor Final'].mean()
    
    #E-mail

    msg= MIMEMultipart()

    msg['Subject'] = f"OnePage - Loja {loja}"
    msg['From'] = 'testes14lii@gmail.com'
    msg['To'] = emails.loc[emails['Loja']==loja,'E-mail'].values[0]
    
    nome = emails.loc[emails['Loja']==loja,'Gerente'].values[0]
            #cor
    #Faturamento Dia
    if ft_dia >= meta_dia:
        cor_fd= 'green'
    else:
        cor_fd= 'red'
    #Faturamento Ano
    if ft_ano >= meta_ano:
        cor_fa= 'green'
    else:
        cor_fa= 'red'
    #Quantidade Dia
    if qtde_dia >= meta_qtde_dia:
        cor_qd= 'green'
    else:
        cor_qd= 'red'
    #Quantidade An0   
    if qtde_ano >= meta_qtde_ano:
        cor_qa= 'green'
    else:
        cor_qa= 'red'
    #Ticket dia
    if tk_dia >= meta_tkmed_dia:
        cor_td= 'green'
    else:
        cor_td= 'red'
    #Ticket ano
    if tk_ano >= meta_tkmed_ano:
        cor_ta= 'green'
    else:
        cor_ta= 'red'

    corpo_email = f""" <p>Bom dia, {nome} </p>

    <p>O Resultado de ontem <b>({dia_indicador.day}/{dia_indicador.month})</b> da <b>loja {loja}</b> foi:</p>

    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenário Dia</th>
      </tr>
      <tr>
        <td style="text-align: center">Faturameto</td>
        <td style="text-align: center">R$ {ft_dia:.2f}</td>
        <td style="text-align: center">R$ {meta_dia:.2f}</td>
        <td style="text-align: center"><font color={cor_fd}>◙</font></td>
      </tr>
      <tr>
        <td style="text-align: center">Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_dia}</td>
        <td style="text-align: center">{meta_qtde_dia}</td>
        <td style="text-align: center"><font color={cor_qd}>◙</font></td>
      </tr>
      <tr>
        <td style="text-align: center">Ticket Médio</td>
        <td style="text-align: center">R$ {tk_dia:.2f}</td>
        <td style="text-align: center">R$ {meta_tkmed_dia:.2f}</td>
        <td style="text-align: center"><font color={cor_td}>◙</font></td>
      </tr>

    </table>
    <br>
    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Ano</th>
        <th>Meta Ano</th>
        <th>Cenário Ano</th>
      </tr>
      <tr>
        <td style="text-align: center">Faturameto</td>
        <td style="text-align: center">R$ {ft_ano:.2f}</td>
        <td style="text-align: center">R$ {meta_ano:.2f}</td>
        <td style="text-align: center"><font color={cor_fa}>◙</font></td>
      </tr>
      <tr>
        <td style="text-align: center">Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_ano}</td>
        <td style="text-align: center">{meta_qtde_ano}</td>
        <td style="text-align: center"><font color={cor_qa}>◙</font></td>
      </tr>
      <tr>
        <td style="text-align: center">Ticket Médio</td>
        <td style="text-align: center">R$ {tk_ano:.2f}</td>
        <td style="text-align: center">R$ {meta_tkmed_ano:.2f}</td>
        <td style="text-align: center"><font color={cor_ta}>◙</font></td>
      </tr>

    </table>

    <p>Segue em anexo a planilha com todos os dados para mais detalhes.</p>

    <p>Qualquer dúvida estou à disposição. </p>
    <p>Att., Isaias</p>

    """
    msg.attach(MIMEText(corpo_email,'html'))
    
    cam_anexo= local_aq
    filename = str(nome_aq)
    attachment = open(cam_anexo, "rb")

    anexo_type = mimetypes.guess_type(f'{filename}')[0].split('/')
    part = MIMEBase(anexo_type[0], anexo_type[1])

    part.set_payload((attachment).read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition',"attachment; filename= %s" % filename)

    msg.attach(part)

    s = smtplib.SMTP('smtp.gmail.com: 587')
    s.starttls()

    #login Credentials for sending the mail
    s.login(msg['From'], senha)
    s.sendmail(msg['From'], msg['To'], msg.as_string())
    s.quit()
    print(f"E-mail da loja {loja} enviado com Sucesso")

E-mail da loja Iguatemi Esplanada enviado com Sucesso
E-mail da loja Shopping Midway Mall enviado com Sucesso
E-mail da loja Norte Shopping enviado com Sucesso
E-mail da loja Shopping Iguatemi Fortaleza enviado com Sucesso
E-mail da loja Shopping União de Osasco enviado com Sucesso
E-mail da loja Shopping Center Interlagos enviado com Sucesso
E-mail da loja Rio Mar Recife enviado com Sucesso
E-mail da loja Salvador Shopping enviado com Sucesso
E-mail da loja Rio Mar Shopping Fortaleza enviado com Sucesso
E-mail da loja Shopping Center Leste Aricanduva enviado com Sucesso
E-mail da loja Ribeirão Shopping enviado com Sucesso
E-mail da loja Shopping Morumbi enviado com Sucesso
E-mail da loja Parque Dom Pedro Shopping enviado com Sucesso
E-mail da loja Bourbon Shopping SP enviado com Sucesso
E-mail da loja Palladium Shopping Curitiba enviado com Sucesso
E-mail da loja Passei das Águas Shopping enviado com Sucesso
E-mail da loja Center Shopping Uberlândia enviado com Sucesso
E-mail da loja 

### 5 - Criar Ranking parra diretoria

In [52]:
ft_lojas = vendas.groupby('Loja')[['Loja','Valor Final']].sum()
ft_lojas_ano = ft_lojas.sort_values(by='Valor Final', ascending= False)
vd(ft_lojas_ano)

nome_aq= f'{dia_indicador.day}_{dia_indicador.month}_{dia_indicador.year}_Ranking Anual.xlsx'   
ft_lojas_ano.to_excel(r'{}\{}'.format(caminho_bk,nome_aq))

vendas_dia = vendas.loc[vendas['Data']==dia_indicador, :]
ft_lojas_dia = vendas_dia.groupby('Loja')[['Loja','Valor Final']].sum()
ft_lojas_dia = ft_lojas_dia.sort_values(by='Valor Final', ascending= False)
vd(ft_lojas_dia)

nome_aq= f'{dia_indicador.day}_{dia_indicador.month}_{dia_indicador.year}_Ranking Dia.xlsx'   
ft_lojas_dia.to_excel(r'{}\{}'.format(caminho_bk,nome_aq)) 

,Valor Final
Loja,
Iguatemi Campinas,1762419
Shopping Vila Velha,1731167
Bourbon Shopping SP,1726110
Rio Mar Recife,1722766
Shopping SP Market,1721763
Palladium Shopping Curitiba,1721120
Norte Shopping,1711968
Ribeirão Shopping,1707122
Iguatemi Esplanada,1699681


,Valor Final
Loja,
Salvador Shopping,3950
Novo Shopping Ribeirão Preto,3400
Center Shopping Uberlândia,2651
Shopping Eldorado,2391
Shopping Center Interlagos,1582
Shopping Recife,1366
Norte Shopping,1259
Shopping União de Osasco,1207
Shopping Vila Velha,937


### 6 - E-mail para diretoria

In [53]:
msg= MIMEMultipart()

msg['Subject'] = f"Ranking do dia {dia_indicador.day}/{dia_indicador.month} "
msg['From'] = 'testes14lii@gmail.com'
msg['To'] = emails.loc[emails['Loja']=='Diretoria','E-mail'].values[0]

nome = emails.loc[emails['Loja']=='Diretoria','Gerente'].values[0]

corpo_email = f""" <p>Bom dia, {nome} </p>

<p>O Raking do dia<b>({dia_indicador.day}/{dia_indicador.month})</b> e do ano de <b>{dia_indicador.year} são:</b></p>

<p>Melhor loja do Dia em Faturamento:<b> Loja {ft_lojas_dia.index[0]} com Faturamento R$ {ft_lojas_dia.iloc[0,0]:.2f} </b></p>
<p>Pior loja do Dia em Faturamento:<b> Loja {ft_lojas_dia.index[-1]} com Faturamento R$ {ft_lojas_dia.iloc[-1,0]:.2f} </b></p>

<p>Melhor loja do Ano em Faturamento:<b> Loja {ft_lojas_ano.index[0]} com Faturamento R$ {ft_lojas_ano.iloc[0,0]:.2f} </b></p>
<p>Pior loja do Ano em Faturamento:<b> Loja {ft_lojas_ano.index[-1]} com Faturamento R$ {ft_lojas_ano.iloc[-1,0]:.2f} </b></p>

<p>Segue em anexo a planilha com todos os dados.</p>

<p>Qualquer dúvida estou à disposição. </p>

<p>Att., Isaias</p>

"""
msg.attach(MIMEText(corpo_email,'html'))
# Anual
filename = f'{dia_indicador.day}_{dia_indicador.month}_{dia_indicador.year}_Ranking Anual.xlsx'
cam_anexo_d= r'{}\{}'.format(caminho_bk,filename)
attachment = open(cam_anexo_d, "rb")

anexo_type = mimetypes.guess_type(f'{filename}')[0].split('/')
part = MIMEBase(anexo_type[0], anexo_type[1])

part.set_payload((attachment).read())
encoders.encode_base64(part)
part.add_header('Content-Disposition',"attachment; filename= %s" % filename)

msg.attach(part)
#Dia
filename = f'{dia_indicador.day}_{dia_indicador.month}_{dia_indicador.year}_Ranking Dia.xlsx'
cam_anexo_d= r'{}\{}'.format(caminho_bk,filename)
attachment = open(cam_anexo_d, "rb")

anexo_type = mimetypes.guess_type(f'{filename}')[0].split('/')
part = MIMEBase(anexo_type[0], anexo_type[1])

part.set_payload((attachment).read())
encoders.encode_base64(part)
part.add_header('Content-Disposition',"attachment; filename= %s" % filename)

msg.attach(part)

s = smtplib.SMTP('smtp.gmail.com: 587')
s.starttls()

#login Credentials for sending the mail
s.login(msg['From'], senha)
s.sendmail(msg['From'], msg['To'], msg.as_string())
s.quit()
print(f"E-mail para Dirretoria enviado com Sucesso")

E-mail para Dirretoria enviado com Sucesso
